# Covariance and contravariance
- covariant: if `Parent ≤ Child` then `I<Parent> ≤ I<Child>`
- contravariant: if `Parent ≤ Child` then `I<Child> ≤ I<Parent>`
- bivariant: if `Parent ≤ Child` then `I<Parent> ≡ I<Child>` (case like `type 'a t = int`)
- variant: if covariant, contravariant or bivariant
- invariant or nonvariant: not variant 

##### 🟢 &nbsp; &nbsp; Covariant: if `Parent ≤ Child` then `I<Parent> ≤ I<Child>` 

In [20]:
type animal = <age : float; color: string> ;;
let dog ~age ~color  = object 
    method age = age
    method color = color
    method bark = "wof wof"
end ;;

(* list is covariant *)
let dogs : animal list = [ (dog ~age:1.1 ~color:"gray" :> animal); (dog ~age:2.1 ~color:"dark" :> animal) ] ;;

let show (animals : animal list) = 
    animals |> List.fold_left (fun acc x -> acc ^ Printf.sprintf " <age=%F, color=%s> " x#age x#color) "";;
    
show dogs ;;

type on_off = [ `On | `Off ] ;;
let on : on_off = `On ;;
let off : on_off = `Off ;;
type tri = [ on_off | `Number of float ] ;;
let tri_list : tri list = [ (on :> tri); (off :> tri); `Number 0.1 ] ;;

type animal = < age : float; color : string >


val dog : age:'a -> color:'b -> < age : 'a; bark : string; color : 'b > =
  <fun>


val cat : < age : float; color : string; meow : string > = <obj>


val dogs : animal list = [<obj>; <obj>]


val show : animal list -> string = <fun>


...

type on_off = [ `Off | `On ]


val on : on_off = `On


val off : on_off = `Off


type tri = [ `Number of float | `Off | `On ]


val tri_list : tri list = [`On; `Off; `Number 0.1]


The following code throws error cause Array in OCaml is, like in Kotlin, invariant unlike C# or Java.

In [16]:
type animal = <age : float; color: string> ;;
let dog ~age ~color  = object 
    method age = age
    method color = color
    method bark = "wof wof"
end ;;

let dogs : animal array = [ (dog ~age:1.1 ~color:"gray" :> animal); (dog ~age:2.1 ~color:"dark" :> animal) ] ;;

type animal = < age : float; color : string >


val dog : age:'a -> color:'b -> < age : 'a; bark : string; color : 'b > =
  <fun>


val cat : < age : float; color : string; meow : string > = <obj>


error: compile_error

<br/>

##### 🟢 &nbsp; &nbsp; Contravariant: if `Parent ≤ Child` then `I<Child> ≤ I<Parent>`

In [40]:
type animal = <age : float; color: string> ;;
type dog = <age: float; color: string; bark : string> ;;
type cat = <age: float; color: string; meow : string> ;;
let create_dog ~age ~color : dog = object 
    method age = age
    method color = color
    method bark = "wof wof"
end ;;

let create_cat ~age ~color : cat = object
    method age = age
    method color = color
    method meow = "meow meow"
end ;;

let show : animal -> string = fun x -> Printf.sprintf "<age=%F, color=%s>" x#age x#color ;;
let bark : dog -> string = fun x -> x#bark ;;
let meow : cat -> string = fun x -> x#meow ;;

(* animal -> string is a subtype of cat -> string or dog -> string *)
let foo1 = (show :> cat -> string) ;;
let foo2 = (show :> dog -> string) ;;

type animal = < age : float; color : string >


type dog = < age : float; bark : string; color : string >


type cat = < age : float; color : string; meow : string >


val create_dog : age:float -> color:string -> dog = <fun>


val create_cat : age:float -> color:string -> cat = <fun>


...

val show : animal_to_string = <fun>


val bark : dog -> string = <fun>


val meow : cat -> string = <fun>


val foo1 : cat -> string = <fun>


val foo2 : dog -> string = <fun>


<br/>

##### 🟢 &nbsp; &nbsp; Variance annotations or constraints

https://stackoverflow.com/questions/70450821/ocaml-meaning-of-in-type-a-t

```module type T = sig type 'a t end```

Above abstract type constructor could be either one of the followings:
- `type 'a t = a * a` (produce or contain `'a` - covariant)
- `type 'a t = 'a -> unit` (consume `'a` - contravariant)
- `type 'a t = int` (ignore its argument all together - bivariant)
- `type 'a t = { get : unit -> 'a; store : 'a -> unit }` (contain a viewable and mutable reference to `'a` - both covariant and contravariant }

OCaml ends up knowing nothing about `'a t`. By default OCaml will assume `'a t` is invariant. So under certain situations, Variance annotations (`+` or `-`) should be given to `'a t`. `+` for covariant and `-` for contravariant.

In [58]:
(* covariant annotation *)

type animal = <age : float; color: string> ;;
type dog = <age: float; color: string; bark : string> ;;
type cat = <age: float; color: string; meow : string> ;;
let create_dog ~age ~color : dog = object 
    method age = age
    method color = color
    method bark = "wof wof"
end ;;

let create_cat ~age ~color : cat = object
    method age = age
    method color = color
    method meow = "meow meow"
end ;;

(* won't work without covariant notation + because without it, OCaml will assume the type is invariant *)
module type ANIMAL_HOUSE = sig
    type (+'e, +'w, +'n, +'s) t 
    val east : 'e -> ('e, _,  _, _) t
    val west : 'w -> (_, 'w, _, _) t
    val north : 'n -> (_, _, 'n, _) t
    val south : 's -> (_, _, _, 's) t
end

module AnimalHouse :  ANIMAL_HOUSE = struct
    type ('e, 'w, 'n, 's) t =
        | East of 'e
        | West of 'w
        | North of 'n
        | South of 's
    let east a = East a
    let west a = West a
    let north a = North a
    let south a = South a
end ;;

let east_cat_house = AnimalHouse.east (create_cat ~age:1.1 ~color:"grey") ;;
(east_cat_house :> (animal, _, _, _) AnimalHouse.t);;

let west_dog_house = AnimalHouse.west (create_dog ~age:1.1 ~color:"grey") ;;
(west_dog_house :> (animal, _, _, _) AnimalHouse.t);;

type animal = < age : float; color : string >


type dog = < age : float; bark : string; color : string >


type cat = < age : float; color : string; meow : string >


val create_dog : age:float -> color:string -> dog = <fun>


val create_cat : age:float -> color:string -> cat = <fun>


...

module AnimalHouse : ANIMAL_HOUSE


val east_cat_house : (cat, 'a, 'b, 'c) AnimalHouse.t = <abstr>


- : (animal, 'a, 'b, 'c) AnimalHouse.t = <abstr>


val west_dog_house : ('a, dog, 'b, 'c) AnimalHouse.t = <abstr>


- : (animal, dog, 'a, 'b) AnimalHouse.t = <abstr>


In [189]:
(* contravariant annotation *)

module type T = sig
    type (-'a, +'b) t 
end ;;

module M : T = struct
    type (-'a, +'b) t = 'a -> 'b
end


module type T = sig type (-'a, +'b) t end


module M : T
